In [ ]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install scikit-learn
%pip install openpyxl
%pip install xgboost
%pip install sklearn.preprocessing
#pip install catboost

In [ ]:
# Import the required packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Google Sheet Link: https://docs.google.com/spreadsheets/d/1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0/edit#gid=0
pd.set_option('display.max_rows', 200, 'display.min_rows', 30) #Displays a max of 200 rows for testing

sheet_id = '1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0'
xls = pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx")

arrivals = pd.read_excel(xls, 'Arrivals', header = 0) #Reads in the Google Sheets Data
departures = pd.read_excel(xls, 'Departures', header = 0)
arrivals = arrivals.drop(['blank', 'Communication Type/Reason for Speaking.1'], axis=1) #Drops unnecessary Columns/Headers
departures = departures.drop(['Airport', 'Communication Type/Reason for Speaking.1'], axis=1)

arrivals = arrivals.dropna(thresh=14) #Drops all rows and columns with more than 14 N/A values
departures = departures.dropna(thresh=14)

# create a new leftmost column for the index
# iterate through the rows and index the responses of each flight based on unique flight no.
counter = 0
flight_no = -1
arrivals.insert(loc = 0, # create arrivals indexing
                column = 'index',
                value = 0)
for row in arrivals.iterrows():
    if flight_no != row[1]['Flight No.']:
        flight_no = row[1]['Flight No.']
        counter = 0
    arrivals.loc[row[0], 'index'] = counter
    counter += 1

counter = 0
flight_no = -1
departures.insert(loc = 0, # create departures indexing
                column = 'index',
                value = 0)
for row in departures.iterrows():
    if flight_no != row[1]['Flight No.']:
        flight_no = row[1]['Flight No.']
        counter = 0
    departures.loc[row[0], 'index'] = counter
    counter += 1


#Gets the specifics columns from collected data
x = arrivals[['index']]
y = arrivals[['Communication Type/Reason for Speaking']]

# Change y-axis labels to numeric values
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y.values.ravel())

#split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = .25, random_state = 42)

import xgboost as xgb

import xgboost as xgb

model = xgb.XGBClassifier()
model.fit(x, y)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, x, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

new_x = pd.DataFrame({'index': [0, 1, 2, 3]})
predictions = model.predict(new_x)